<a href="https://colab.research.google.com/github/mvsamudrala/PLAIG/blob/main/PLAIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

--2024-11-12 18:55:34--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148337011 (141M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 141.46M   135MB/s    in 1.1s    

2024-11-12 18:55:35 (135 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [148337011/148337011]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONP

In [ ]:
!which conda

/usr/local/bin/conda


In [ ]:
!conda --version
!python --version

conda 24.9.2
Python 3.12.7


In [ ]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/root/.ipython',
 '/usr/local/lib/python3.10/site-packages']

In [ ]:
sys.path.append('/usr/local/lib/python3.10/site-packages')

In [ ]:
!wget https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/plaig_environment.yml -O environment.yml

--2024-11-12 19:16:45--  https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/plaig_environment.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 427 [text/plain]
Saving to: ‘environment.yml’

environment.yml     100%[===================>]     427  --.-KB/s    in 0s      

2024-11-12 19:16:45 (7.24 MB/s) - ‘environment.yml’ saved [427/427]



In [ ]:
!conda env create -n plaig_venv -f environment.yml

/usr/local/lib/python3.12/argparse.py:2006: FutureWarning: `remote_definition` is deprecated and will be removed in 25.9. Use `conda env create --file=URL` instead.
  action(self, namespace, argument_values, option_string)
Channels:
 - defaults
Platform: linux-64
Solving environment: \ | / done


Preparing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
Verifying transaction: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [ ]:
!conda activate aec_venv


CondaError: Run 'conda init' before 'conda activate'



In [ ]:
!conda env remove -n plaig_venv -y



Remove all packages in environment /usr/local/envs/plaig_venv:


## Package Plan ##

  environment location: /usr/local/envs/plaig_venv


The following packages will be REMOVED:

  _libgcc_mutex-0.1-main
  _openmp_mutex-5.1-1_gnu
  biopython-1.78-py310h7f8727e_0
  blas-1.0-mkl
  bottleneck-1.4.2-py310ha9d4c09_0
  brotli-1.0.9-h5eee18b_8
  brotli-bin-1.0.9-h5eee18b_8
  bzip2-1.0.8-h5eee18b_6
  ca-certificates-2024.9.24-h06a4308_0
  contourpy-1.2.0-py310hdb19cb5_0
  cycler-0.11.0-pyhd3eb1b0_0
  cyrus-sasl-2.1.28-h52b45da_1
  dbus-1.13.18-hb2f20db_0
  et_xmlfile-1.1.0-py310h06a4308_0
  expat-2.6.3-h6a678d5_0
  filelock-3.13.1-py310h06a4308_0
  fontconfig-2.14.1-h55d465d_3
  fonttools-4.51.0-py310h5eee18b_0
  freetype-2.12.1-h4a9f257_0
  fsspec-2024.6.1-py310h06a4308_0
  glib-2.78.4-h6a678d5_0
  glib-tools-2.78.4-h6a678d5_0
  gmp-6.2.1-h295c915_3
  gmpy2-2.1.2-py310heeb90bb_0
  gst-plugins-base-1.14.1-h6a678d5_1
  gstreamer-1.14.1-h5eee18b_1
  icu-73.1-h6a678d5_0
  intel-openmp-2023.1.0-h

In [ ]:
!conda install pandas numpy openpyxl scipy matplotlib scikit-learn pip networkx pytorch scipy biopython xgboost -y
!pip install biopandas rdkit torch-geometric

Channels:
 - defaults
Platform: linux-64
Solving environment: | / - \ done

# All requested packages already installed.

  Using cached biopandas-0.5.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached mmtf_python-1.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached looseversion-1.1.2-py3-none-any.whl.metadata (4.6 kB)
  Using cached psutil-6.1.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
Using cached biopandas-0.5.1-py3-none-any.whl (68 kB)
Using cached looseversion-1.1.2-py3-none-any.whl (8.1 kB)
Using cached mmtf_python-1.1.3-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.7/32.7 MB 89.2 MB/s eta 0:00

In [ ]:
!wget https://github.com/mvsamudrala/PLAIG/raw/refs/heads/main/binana.zip -O binana.zip
!unzip binana.zip

--2024-11-12 19:34:11--  https://github.com/mvsamudrala/PLAIG/raw/refs/heads/main/binana.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/binana.zip [following]
--2024-11-12 19:34:11--  https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/binana.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 946382 (924K) [application/zip]
Saving to: ‘binana.zip’

binana.zip          100%[===================>] 924.20K  --.-KB/s    in 0.06s   

2024-11-12 19:34:11 (15.4 MB/s) - ‘binana.zip’ saved [946382/946382]

Archive:  binana.zip
   creating: binana/
  

In [ ]:
import binana

In [ ]:
!wget https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/PLAIG_Run.py
!wget https://github.com/mvsamudrala/PLAIG/blob/main/PLAIG_Architecture.py
!wget https://github.com/mvsamudrala/PLAIG/raw/refs/heads/main/PLAIG_Model.pth
!wget https://github.com/mvsamudrala/PLAIG/raw/refs/heads/main/PLAIG_Stacking.joblib.zip
!unzip PLAIG_Stacking.joblib.zip

--2024-11-12 19:52:33--  https://raw.githubusercontent.com/mvsamudrala/PLAIG/refs/heads/main/PLAIG_Run.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7512 (7.3K) [text/plain]
Saving to: ‘PLAIG_Run.py’

PLAIG_Run.py        100%[===================>]   7.34K  --.-KB/s    in 0s      

2024-11-12 19:52:34 (67.3 MB/s) - ‘PLAIG_Run.py’ saved [7512/7512]

--2024-11-12 19:52:34--  https://github.com/mvsamudrala/PLAIG/blob/main/PLAIG_Architecture.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘PLAIG_Architecture.py’

PLAIG_Architecture.     [ <=>                ] 341.36K  --.-KB/s    in 0.08s   

202

In [ ]:
ls

binana/     Miniconda3-latest-Linux-x86_64.sh*  PLAIG_Run.py               sample_data/
binana.zip  PLAIG_Architecture.py               PLAIG_Stacking.joblib
__MACOSX/   PLAIG_Model.pth                     PLAIG_Stacking.joblib.zip


In [ ]:
# %%capture
start = time.time()
normalization_statistics_file = "combined_set_normalization_statistics.pkl"
with open(normalization_statistics_file, 'rb') as file:
    normalization_statistics = pickle.load(file)
# graph_data_file = "DUDE_Z_norm_3.pkl"
# with open(graph_data_file, 'rb') as file:
#     dataset = pickle.load(file)
# print(len(dataset))
warnings.filterwarnings('ignore')
cannot_read_mols = []
count = 0
distance_cutoff = 3
pre_dataset = []
all_ligand_features = []
all_pocket_features = []
# all_ligand_features_key = {}
# all_pocket_features_key = {}
dataset = []
no_nodes_count = 0
for protein_pdb, protein_pdbqt, ligand_pdb, ligand_pdbqt in complex_files():
    # random.shuffle(ligands)
    count = 0
    protein_name = os.path.splitext(os.path.basename(protein))[0]
    print(protein_name)
    protein_pocket_path = protein_pdb
    print(protein_pocket_path)
    protein_pocket_pdbqt_path = protein_pdbqt
    print(protein_pocket_pdbqt_path)
    # protein_path = "/Users/mvsamudrala/BindingAffinityGNN/refined-set/1a1e/1a1e_protein.pdb"
    # pdbqt_output_dir = "/Users/mvsamudrala/BindingAffinityGNN/PDBQT_refined"
    # path_parts = protein_pocket_path.split("/")
    # pdb_code = path_parts[-2]
    ligand_name = os.path.splitext(os.path.basename(ligand_pdb))[0]
    print(ligand_name)
    ligand_path = ligand_pdb
    print(ligand_path)
    ligand_pdbqt_path = ligand_pdbqt
    print(ligand_pdbqt_path)
    print()
    try:
        graph = graph_representation.pl_complex_to_graph(protein_pocket_path, ligand_path, protein_pocket_pdbqt_path, ligand_pdbqt_path, distance_cutoff)
        # for edge in graph.edges(data=True):
        #     print(edge)
        if len(graph.nodes) == 0:
            no_nodes_count += 1
            print(f"Graph has no nodes, #{no_nodes_count}")
            continue
        pre_dataset.append((graph, (protein_name, ligand_name)))
        # all_ligand_features_key[code] = graph.graph["ligand_attr"]
        # all_pocket_features_key[code] = graph.graph["pocket_attr"]
        print(graph.graph["ligand_attr"])
        all_ligand_features.append(graph.graph["ligand_attr"])
        print(graph.graph["pocket_attr"])
        all_pocket_features.append(graph.graph["pocket_attr"])
        # in_channels = graph_data.num_node_features
        # edge_channels = graph_data.edge_features.shape[1]
        # hidden_channels = 32
        # out_channels = 1
        # num_graph_features = len(graph_data.graph_features)
        #
        # model = GCN(in_channels, edge_channels, hidden_channels, out_channels, num_graph_features)
        #
        # print(model)

    except Exception as e:
        cannot_read_mols.append((protein_name, ligand_name))
        print(f"Cannot read {protein_name}, {ligand_name} file: {e}")
    # if count == 1000:
    #     break
    count += 1
print()
all_ligand_features_normalized, all_pocket_features_normalized = PLAIG_Run.normalize_graph_features(all_ligand_features, all_pocket_features, normalization_statistics)
for index, (graph, (protein_name, ligand_name)) in enumerate(pre_dataset):
    graph.graph["ligand_attr"] = all_ligand_features_normalized[index]
    print(graph.graph["ligand_attr"])
    graph.graph["pocket_attr"] = all_pocket_features_normalized[index]
    print(graph.graph["pocket_attr"])
    graph_data = PLAIG_Run.prepare_data(graph, f"{protein_name}, {ligand_name}")
    # count = 0
    # for edge in graph.edges():
    #     print(count)
    #     print(edge)
    #     count += 1
    print(f"Data object: {graph_data}")
    print(f'Number of nodes: {graph_data.num_nodes}')
    print(f'Number of edges: {graph_data.num_edges}')
    print(f'Average node degree: {graph_data.num_edges / graph_data.num_nodes:.2f}')
    print(f'Has isolated nodes: {graph_data.has_isolated_nodes()}')
    print(f'Has self-loops: {graph_data.has_self_loops()}')
    print(f'Is undirected: {graph_data.is_undirected()}')
    print()
    dataset.append(graph_data)
print(len(dataset))
# graph_data_file = "DUDE_Z_norm_3_pt2.pkl"
# with open(graph_data_file, 'wb') as file:
#     pickle.dump(dataset, file)

Input your .PDB and .PDBQT protein and ligand files for preprocessing.